# Werkzeuge zum Auffinden von Peaks und Perioden in Datenreihen

Mit den folgenden Code-Fragmenten zeigen wir Ihnen: 

 * Wie man die **Autokorrelationsfunktion** einer Datenreihe mit der Funktion `autocorrelate` aus PhyPraKit berechnen kann.
 * Wie man mit der *scipy* Funktion `find_peak_cwt` Maxima und Minima in Verteilungen finden kann.
 * Wie man mit der Funktion `convolutionPeakfinder`aus PhyPraKit Maxima und Minima in Verteilungen finden kann.

Alle im Folgenden gezeigten Code-Fragmente lassen sich geeignet kombinieren. Dabei können Sie selbst ein Gefühl dafür bekommen, wie zuverlässig durch die entsprechenden Funktionen Maxima und Minima in Datenreihen gefunden werden und wie hilfreich die Autokorrelation von Datenreihen sein kann, um Periodizitäten besser identifizieren zu können. 

Dazu verwenden wir die Beispiel-Datei `AudioData.csv`, aus dem *tools*-Verzeichnis, die periodische Daten enthält. 

## Einlesen der Daten im *csv*-Format mit *pandas*

Wie man *csv*-Dateien mit *pandas* **einließt** finden Sie z.B. [hier](https://gitlab.kit.edu/kit/etp-lehre/p1-praktikum/students/-/blob/main/Ferromagnetische_Hysterese/tools/hysteresis_curve.ipynb). Für alle weiteren Zellen verwenden wir die Beispiel-Datei `AudioData.csv`, aus dem *tools*-Verzeichnis, die periodische Daten enthält.

In [ ]:
import pandas as pd
# Read cvs file as pandas dataframe
df = pd.read_csv("AudioData.csv")
# Translate dataframe columns into native python lists
t = df["Time (ms)"].to_list()
d = df["Recording (a.u.)"].to_list() 

# Plot the data
import matplotlib.pyplot as plt
fig = plt.figure(1, figsize=(6.0, 6.0))
plt.plot(t, d, color="darkblue", label="Input Data")
plt.xlabel("t (ms)")
plt.ylabel("Signal (AU)")
plt.legend(numpoints=1, loc="best")
plt.grid(linestyle="dashed")
plt.show()

## Einlesen der Daten im *csv*-Format mit PhyPraKit

Wie man die *csv*-Dateien aus dem PicoScope mit der PhyPrakit Funktion *readPicoScope* **einließt** finden Sie z.B. [hier](https://gitlab.kit.edu/kit/etp-lehre/p1-praktikum/students/-/blob/main/Ferromagnetische_Hysterese/tools/hysteresis_curve.ipynb).

## Smoothing und down sampling mit PhyPraKit

Wie Sie Datenreihen ***down samplen* und glätten** können finden Sie z.B. [hier](https://gitlab.kit.edu/kit/etp-lehre/p1-praktikum/students/-/blob/main/Ferromagnetische_Hysterese/tools/hysteresis_curve.ipynb). 

## Suche nach Autokorrelationen in den Daten mit PhyPrakit

 * Bei periodischen Daten bietet es sich manchmal an die Periodizität mit Hilfe einer **Autokorrelationsfunktion** etwas besser herauszuarbeiten.
 * Mit dem folgenden Code-Fragment zeigen wir Ihnen, wie Sie dies mit der PhyPraKit Funktion *autocorrelate* tun können. 

In [ ]:
import numpy as np
from PhyPraKit.phyTools import autocorrelate
# Find auto correlation
ac_d = autocorrelate(d)

# Plot the auto correlation on top of the original data
fig = plt.figure(2, figsize=(6.0, 6.0))
plt.plot(t, ac_d, color="darkblue", label="Auto correlation")
plt.plot(t, d, color="darkred", label="Input Data")
plt.xlabel("t (ms)")
plt.ylabel("Auto correlation")
plt.legend(numpoints=1, loc="best")
plt.grid(linestyle="dashed")
plt.show()

## Suche nach Maxima und Minima in den Daten mit *scipy*

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie man mit der scipy Klasse [signal.find_peaks_cwt](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks_cwt.html#scipy.signal.find_peaks_cwt) **Maxima** und **Minima** in einer Datenreihe finden kann.

In [73]:
from scipy.signal import find_peaks_cwt
# Return indices of peaks in auto correlation function, width refers to the 
# window to define a local extremum. It might require a bit of tuning 
pidxac = find_peaks_cwt( ac_d, widths=3)
# Return indices of dips in auto correlation function 
didxac = find_peaks_cwt(-ac_d, widths=3)

## Suche nach Maxima und Minima in den Daten mit PhyPraKit

Mit dem folgenden Code-Fragment zeigen wir Ihnen:
    
 * Wie Sie das gleiche mit der Funktion `convolutionPeakfinder` aus PhyPraKit erreichen.

In [69]:
from PhyPraKit.phyTools import convolutionPeakfinder

# find maxima and minima using convolution peak finder
width = 3
pidxac = convolutionPeakfinder( ac_d, width, th=0.5)
didxac = convolutionPeakfinder(-ac_d, width, th=0.5)

## Darstellung der Maxima und Minima mit *matplotlib*

Mit dem folgenden Code-Fragment demonstrieren wir, das die Minima und Maxima erfolgreich gefunden wurden.

In [ ]:
# Prepare arrays for plotting
if len(pidxac) > 1:
    # The first peak is at 0 by construction. If you let the peak finder run 
    # loos it can stumble at the array edges, since it averages over a number 
    # of "widths" indices to determine the peak position
    pidxac[0] = 0  
    ac_tp, ac_dp = np.array(t)[pidxac], np.array(ac_d)[pidxac]
    ac_td, ac_dd = np.array(t)[didxac], np.array(ac_d)[didxac]
else:
    print("Not enough correlation peaks found.")

# Plot the identified minima and maxima on top of auto correlation
fig = plt.figure(3, figsize=(6.0, 6.0))
plt.plot(t, ac_d, color="darkblue", label="Auto correlation")
plt.plot(ac_tp, ac_dp, "gx", label="Identified maxima")
plt.plot(ac_td, ac_dd, "rx", label="Identified minima")
plt.xlabel("t (ms)")
plt.ylabel("Auto correlation")
plt.legend(numpoints=1, loc="best")
plt.grid(linestyle="dashed")
plt.show()

## Check

Führen Sie zur Wiederholung des Gelernten und zum Test Ihres Verständnisses noch einmal die folgenden Operationen durch:

 * Versuchen Sie die Maxima und Minima der Datenreihe selbst zu bestimmen.
 * Berechnen Sie Werte für $\Delta t$ aus den Differenzen der Maxima aus der Datenreihe selbst.
 * Vergleichen Sie die berechneten Werte für $\Delta t$ aus der Datenreihe selbst und aus der Autokorrelationsfunktion. 

In [ ]:
# ADD YOUR CODE BELOW